In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
#from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
from config import g_key
import traceback
import gmaps
import gmaps.datasets
from geopy.distance import geodesic

In [2]:
#Helper function to convert the 3 element tuple to just lat,long tuple
def convert_coord_to_latlng(coord):
    return (coord[1],coord[2])

In [3]:
#Read the CSV into a dataframe
weather_df = pd.read_csv("analysis/WeatherPy_vacation.csv") 

In [4]:
weather_df.head()

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,isangel,vu,78.89,light rain,-19.55,169.27,Volcano Sea View Tree House
1,28,bumba,cd,74.26,light rain,2.18,22.47,Hôtel Cape Town
2,40,bontang,id,75.96,light rain,0.13,117.50,Okra Leaf Guest House Syariah
3,103,port lincoln,au,63.25,light rain,-34.73,135.87,Port Lincoln Hotel
4,109,vao,nc,76.24,light rain,-22.67,167.48,Hotel Kou-Bugny


In [5]:
#Configure GMaps
gmaps.configure(api_key=g_key)
#Dictionary to capture each country name, along with a list of cities that are less than 1000 miles apart.
country_names= {}
coords=[]
#ITerate through each row
for index, row in weather_df.iterrows():
    city = row["City"]
    country = row["Country"]
    lat_long = (city,row["Lat"],row["Lng"],row["Current Description"],row["Hotel Name"],country,row["Max Temp"])
    
    #Initialize the country in the dictionary initially
    if country not in country_names:
        country_names[country]=[]
    if len(country_names[country]) > 0:
        #Check to see if we have already included this city
        has_city = False
        for c in country_names[country]:
            if city == c[0]:
                #Duplicate city so skip
                has_city = True
                break
        if has_city:
            #If that city already exists then continue main loop
            continue
        #Get the distance from the last coordinate pair to this pair
        prev_lat_long = (country_names[country][-1][1],country_names[country][-1][2])
        new_lat_long = (lat_long[1],lat_long[2])
        #User geodeslc to calculate the distance between 2 pairs of lat,long
        distance = geodesic(new_lat_long,prev_lat_long).miles
        #Set max driving distance between cities to 1000 miles
        if distance<1000:
            country_names[country].append(lat_long) 
            #Once we have obtained 4 cities that match the criteria, break out of the loop
            if len(country_names[country]) == 4:
                coords=country_names[country]
                break
    else:
        #As this is the first time adding the country into the dictionary, add that lat_long.
        country_names[country].append(lat_long)

In [6]:
def convert_coord_to_dict(coord):
    return {
        "Hotel Name": coord[4],
        "City": coord[0],
        "Country": coord[5],
        "Current Description": coord[3],
        "Max Temp": coord[6]
    }

In [10]:
#Ensure the above loop was able to calculate at least 4 valid coordinate pairs
if len(coords) == 4:
    #Sort the coordinates list of tuples by latitude 
    coords.sort(key=lambda x:x[1])
    start_coord = convert_coord_to_latlng(coords[0])
    end_coord = convert_coord_to_latlng(coords[3])
    wp1 = convert_coord_to_latlng(coords[1])
    wp2 = convert_coord_to_latlng(coords[2])
    
    #Cetner the figure around one of the coordinates and zoom in
    fig = gmaps.figure(center=wp1, zoom_level=5)
    clean_coords = [start_coord,wp1,wp2,end_coord]
    #Set the directions mode to Driving directions.
    travel_mode_str='DRIVING'
    #Get the driving directions layer from the Directions API
    gmaps_dirs = gmaps.directions_layer(start_coord, end_coord, waypoints=[wp1,wp2],travel_mode=travel_mode_str,stroke_color='red',stroke_weight=3.0,stroke_opacity=1.0,show_markers=False)
    fig.add_layer(gmaps_dirs)
    
    #Set HTML template for the popup on each marker on GMaps
    info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Descritpion</dt><dd>Description: {Current Description} with a max temp: {Max Temp} °F</dd>
    </dl>
    """
    #Convert into a dictionary so the HTML can parse it.
    c_dicts=[]
    for c in coords:
        c_dict = convert_coord_to_dict(c)
        c_dicts.append(c_dict)
    # Store the DataFrame Row.
    hotel_info = [info_box_template.format(**c) for c in c_dicts]
    marker_layer = gmaps.marker_layer(clean_coords, info_box_content=hotel_info)
    #Apply the layers on the GMap
    fig.add_layer(marker_layer)
    
fig

Figure(layout=FigureLayout(height='420px'))